In [15]:
%pylab inline
import gc
import galsim
import numpy as np
import astropy.io.fits as pyfits

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd wrkDir/FPFS2/sim

/home/xiangchong/work/gw_ipmu/work/FPFS2/sim


In [17]:
ifield      =   0
ngrid       =   64
nx          =   8
ny          =   nx
ndata       =   nx*ny
nrot        =   2
scale       =   0.168
bigfft      =   galsim.GSParams(maximum_fft_size=10240)
flux_scaling=   2.587

variance    =   0.0035
ud          =   galsim.UniformDeviate(ifield*10000+1)
np.random.seed(ifield*10000+1)

# training data
catName     =   'real_galaxy_catalog_25.2.fits'
directory   =   '../../galsim_train/COSMOS_25.2_training_sample/'
cosmos_cat  =   galsim.COSMOSCatalog(catName, dir=directory)

# index
index_use   =   cosmos_cat.orig_index
# parametric catalog
param_cat   =   cosmos_cat.param_cat[index_use]
ngAll       =   len(index_use)

# Get the psf and nosie information
psfFname    =   os.path.join('psfPre','psf%04d.fits'%(ifield))
psfImg      =   galsim.fits.read(psfFname)

psfInt      =   galsim.InterpolatedImage(psfImg,scale=scale,flux = 1.)
g2          =   0.

outFname1   =   'gal%04d-00.fits' %(ifield)
outFname2   =   'gal%04d-20.fits' %(ifield)
gal_image1  =   galsim.ImageF(nx*ngrid,ny*ngrid,scale=scale)
gal_image1.setOrigin(0,0)
gal_image2  =   galsim.ImageF(nx*ngrid,ny*ngrid,scale=scale)
gal_image2.setOrigin(0,0)

data_rows   =   []
i           =   0
while i <ndata:
    # Prepare the subimage
    ix      =   i%nx
    iy      =   i//nx
    b       =   galsim.BoundsI(ix*ngrid,(ix+1)*ngrid-1,iy*ngrid,(iy+1)*ngrid-1)
    sub_image1 = gal_image1[b]
    sub_image2 = gal_image2[b]
    #simulate the galaxy
    if i%nrot==0:
        # update galaxy
        index   =   np.random.randint(0,ngAll)
        ss      =   param_cat[index]

        # prepare the galaxies
        gal0    =   cosmos_cat.makeGalaxy(gal_type='parametric',index=index,gsparams=bigfft)
        gal0    *=  flux_scaling

        # rotate the galaxy
        ang     =   ud()*2.*np.pi * galsim.radians
        gal0    =   gal0.rotate(ang)
    else:
        gal0    =   gal0.rotate(1./nrot*np.pi*galsim.radians)
    gal         =   gal0.shear(g1=0.,g2=0.)
    final       =   galsim.Convolve([psfInt,gal],gsparams=bigfft)
    final.drawImage(sub_image1,method='no_pixel')
    gal         =   gal0.shear(g1=0.02,g2=0.)
    final       =   galsim.Convolve([psfInt,gal],gsparams=bigfft)
    final.drawImage(sub_image2,method='no_pixel')
    #row=(i,index,ss['IDENT'],ss['RA'],ss['DEC'],ss['MAG'],iparent)
    #data_rows.append(row)
    i   +=  1
    del gal,final
    gc.collect
pyfits.writeto(outFname1,gal_image1.array)
pyfits.writeto(outFname2,gal_image2.array)
del gal_image1,gal_image2
gc.collect()

178

In [11]:
ss.dtype.names

('IDENT',
 'mag_auto',
 'flux_radius',
 'zphot',
 'sersicfit',
 'bulgefit',
 'fit_status',
 'fit_mad_s',
 'fit_mad_b',
 'fit_dvc_btt',
 'use_bulgefit',
 'viable_sersic',
 'hlr',
 'flux')